In [ ]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

import os
import sys
import time
import datetime
import argparse
import easydict

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

parser = argparse.ArgumentParser()
# parser.add_argument("--epochs", type=int, default=20, help="number of epochs")
# parser.add_argument("--image_folder", type=str, default="data/artifacts/images", help="path to dataset")
# parser.add_argument("--batch_size", type=int, default=16, help="size of each image batch")
# parser.add_argument("--model_config_path", type=str, default="config/yolov3.cfg", help="path to model config file")
# parser.add_argument("--data_config_path", type=str, default="config/coco.data", help="path to data config file")
# parser.add_argument("--weights_path", type=str, default="config/yolov3.weights", help="path to weights file")
# parser.add_argument("--class_path", type=str, default="config/coco.names", help="path to class label file")
# parser.add_argument("--conf_thres", type=float, default=0.8, help="object confidence threshold")
# parser.add_argument("--nms_thres", type=float, default=0.4, help="iou thresshold for non-maximum suppression")
# parser.add_argument("--n_cpu", type=int, default=0, help="number of cpu threads to use during batch generation")
# parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
# parser.add_argument("--checkpoint_interval", type=int, default=1, help="interval between saving model weights")
# parser.add_argument("--checkpoint_dir", type=str, default="checkpoints", help="directory where model checkpoints are saved")
# parser.add_argument("--use_cuda", type=bool, default=True, help="whether to use cuda if available")
# opt = parser.parse_args()
args = parser.parse_args(args=[])
args = easydict.EasyDict({
    "epochs": 20,
    "image_folder": r"D:/Ivan/Test_data/YoloV3_Data/Yolo_training/vehicles/images",
    "batch_size": 8,
    "model_config_path": "config/yolov3.cfg",
    "data_config_path": "config/coco.data",
    "weights_path": "config/19.weights",
    "class_path": "config/coco.names",
    "conf_thres": 0.8,
    "nms_thres": 0.4,
    "n_cpu": 0,
    "img_size": 416,
    "checkpoint_interval": 1,
    "checkpoint_dir": "checkpoints",
    "use_cuda": True
})

opt=args
print(opt)

cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs("checkpoints", exist_ok=True)

classes = load_classes(opt.class_path)

# Get data configuration
data_config = parse_data_config(opt.data_config_path)
train_path = data_config["train"]
# Get hyper parameters
hyperparams = parse_model_config(opt.model_config_path)[0]
learning_rate = float(hyperparams["learning_rate"])
momentum = float(hyperparams["momentum"])
decay = float(hyperparams["decay"])
burn_in = int(hyperparams["burn_in"])

# Initiate model
model = Darknet(opt.model_config_path)
model.load_weights(opt.weights_path)
#model.apply(weights_init_normal)

if cuda:
    model = model.cuda()

model.train()
# Get dataloader
dataloader = torch.utils.data.DataLoader(
    ListDataset(train_path), batch_size=opt.batch_size, shuffle=False, num_workers=opt.n_cpu
)
print(train_path)
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

for epoch in range(opt.epochs):
    for batch_i, (_, imgs, targets) in enumerate(dataloader):
        
        imgs = Variable(imgs.type(Tensor))
        targets = Variable(targets.type(Tensor), requires_grad=False)
        optimizer.zero_grad()

        loss = model(imgs, targets)

        loss.backward()
        optimizer.step()

        print(
            "[Epoch %d/%d, Batch %d/%d] [Losses: x %f, y %f, w %f, h %f, conf %f, cls %f, total %f, recall: %.5f, precision: %.5f]"
            % (
                epoch,
                opt.epochs,
                batch_i,
                len(dataloader),
                model.losses["x"],
                model.losses["y"],
                model.losses["w"],
                model.losses["h"],
                model.losses["conf"],
                model.losses["cls"],
                loss.item(),
                model.losses["recall"],
                model.losses["precision"],
            )
        )

        model.seen += imgs.size(0)

    if epoch % opt.checkpoint_interval == 0:
        model.save_weights("%s/%d.weights" % (opt.checkpoint_dir, epoch))

{'epochs': 20, 'image_folder': 'D:/Ivan/Test_data/YoloV3_Data/Yolo_training/vehicles/images', 'batch_size': 8, 'model_config_path': 'config/yolov3.cfg', 'data_config_path': 'config/coco.data', 'weights_path': 'config/19.weights', 'class_path': 'config/coco.names', 'conf_thres': 0.8, 'nms_thres': 0.4, 'n_cpu': 0, 'img_size': 416, 'checkpoint_interval': 1, 'checkpoint_dir': 'checkpoints', 'use_cuda': True}


C:\Users\AIC-WS1\Anaconda3\lib\site-packages\torch\nn\_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


img_files D:/Ivan/Test_data/YoloV3_Data/Yolo_training/vehicles/images/00011.jpg

label_files: D:/Ivan/Test_data/YoloV3_Data/Yolo_training/vehicles/labels/00011.txt

D:/Ivan/Test_data/YoloV3_Data/Yolo_training/vehicles/train.txt


C:\Users\AIC-WS1\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


[Epoch 0/20, Batch 0/459] [Losses: x 0.204367, y 0.245743, w 0.080020, h 0.191492, conf 1.206916, cls 0.436563, total 2.365102, recall: 0.12500, precision: 0.00102]
[Epoch 0/20, Batch 1/459] [Losses: x 0.283890, y 0.283446, w 0.149148, h 0.074563, conf 0.734102, cls 0.485393, total 2.010542, recall: 0.00000, precision: 0.00000]
[Epoch 0/20, Batch 2/459] [Losses: x 0.364982, y 0.166702, w 0.079425, h 0.136835, conf 0.725762, cls 0.483526, total 1.957232, recall: 0.00000, precision: 0.00000]
[Epoch 0/20, Batch 3/459] [Losses: x 0.206082, y 0.221480, w 0.036221, h 0.035999, conf 0.934541, cls 0.481543, total 1.915866, recall: 0.00000, precision: 0.00000]
[Epoch 0/20, Batch 4/459] [Losses: x 0.208649, y 0.176830, w 0.089588, h 0.038058, conf 0.770409, cls 0.477661, total 1.761195, recall: 0.00000, precision: 0.00000]
[Epoch 0/20, Batch 5/459] [Losses: x 0.206169, y 0.241500, w 0.151252, h 0.121365, conf 0.758497, cls 0.476672, total 1.955455, recall: 0.00000, precision: 0.00000]
[Epoch 0/2

[Epoch 0/20, Batch 49/459] [Losses: x 0.194406, y 0.308329, w 0.023976, h 0.074956, conf 0.463869, cls 0.143877, total 1.209413, recall: 0.95833, precision: 0.01323]
[Epoch 0/20, Batch 50/459] [Losses: x 0.185114, y 0.216181, w 0.050849, h 0.121584, conf 0.676690, cls 0.170423, total 1.420840, recall: 0.91667, precision: 0.01269]
[Epoch 0/20, Batch 51/459] [Losses: x 0.268422, y 0.221146, w 0.034677, h 0.073642, conf 0.883745, cls 0.198942, total 1.680574, recall: 0.79167, precision: 0.01090]
[Epoch 0/20, Batch 52/459] [Losses: x 0.263749, y 0.301647, w 0.047059, h 0.108662, conf 0.650565, cls 0.172485, total 1.544168, recall: 0.91667, precision: 0.01311]
[Epoch 0/20, Batch 53/459] [Losses: x 0.277210, y 0.301256, w 0.053632, h 0.062503, conf 0.494741, cls 0.159450, total 1.348791, recall: 0.95833, precision: 0.01296]
[Epoch 0/20, Batch 54/459] [Losses: x 0.212815, y 0.238304, w 0.026344, h 0.109532, conf 0.479633, cls 0.172777, total 1.239404, recall: 0.87500, precision: 0.01222]
[Epo

[Epoch 0/20, Batch 98/459] [Losses: x 0.186779, y 0.246562, w 0.293861, h 0.126942, conf 2.318510, cls 0.165072, total 3.337725, recall: 0.79167, precision: 0.01296]
[Epoch 0/20, Batch 99/459] [Losses: x 0.196230, y 0.311939, w 0.067584, h 0.123961, conf 0.392389, cls 0.119462, total 1.211565, recall: 1.00000, precision: 0.01623]
[Epoch 0/20, Batch 100/459] [Losses: x 0.171470, y 0.203443, w 0.060742, h 0.111077, conf 0.380341, cls 0.119543, total 1.046617, recall: 1.00000, precision: 0.01588]
[Epoch 0/20, Batch 101/459] [Losses: x 0.239827, y 0.290688, w 0.060425, h 0.075984, conf 0.364889, cls 0.118793, total 1.150606, recall: 1.00000, precision: 0.01588]
[Epoch 0/20, Batch 102/459] [Losses: x 0.175181, y 0.214107, w 0.057246, h 0.065851, conf 0.869324, cls 0.150478, total 1.532186, recall: 0.87500, precision: 0.01532]
[Epoch 0/20, Batch 103/459] [Losses: x 0.123637, y 0.280870, w 0.048896, h 0.105335, conf 0.457951, cls 0.127043, total 1.143732, recall: 0.95833, precision: 0.01600]


[Epoch 0/20, Batch 147/459] [Losses: x 0.256905, y 0.227188, w 0.025970, h 0.057918, conf 0.338104, cls 0.118070, total 1.024154, recall: 1.00000, precision: 0.02049]
[Epoch 0/20, Batch 148/459] [Losses: x 0.178080, y 0.258478, w 0.041942, h 0.064726, conf 0.355238, cls 0.119147, total 1.017612, recall: 1.00000, precision: 0.02297]
[Epoch 0/20, Batch 149/459] [Losses: x 0.199349, y 0.337955, w 0.059380, h 0.035322, conf 0.691756, cls 0.125431, total 1.449193, recall: 0.95833, precision: 0.02097]
[Epoch 0/20, Batch 150/459] [Losses: x 0.335681, y 0.164774, w 0.026975, h 0.053683, conf 1.227610, cls 0.151066, total 1.959789, recall: 0.87500, precision: 0.02135]
[Epoch 0/20, Batch 151/459] [Losses: x 0.253952, y 0.226355, w 0.023936, h 0.226938, conf 0.438108, cls 0.118222, total 1.287511, recall: 0.95833, precision: 0.02204]
[Epoch 0/20, Batch 152/459] [Losses: x 0.189713, y 0.170485, w 0.016380, h 0.084512, conf 0.341214, cls 0.118463, total 0.920768, recall: 1.00000, precision: 0.02223

[Epoch 0/20, Batch 196/459] [Losses: x 0.140641, y 0.294038, w 0.009414, h 0.057930, conf 0.470332, cls 0.118623, total 1.090979, recall: 1.00000, precision: 0.01603]
[Epoch 0/20, Batch 197/459] [Losses: x 0.146399, y 0.226159, w 0.019939, h 0.092259, conf 2.046735, cls 0.161245, total 2.692735, recall: 0.87500, precision: 0.01393]
[Epoch 0/20, Batch 198/459] [Losses: x 0.142641, y 0.319424, w 0.081258, h 0.154622, conf 1.594067, cls 0.163908, total 2.455920, recall: 0.79167, precision: 0.01250]
[Epoch 0/20, Batch 199/459] [Losses: x 0.137645, y 0.456878, w 0.020386, h 0.025334, conf 0.541637, cls 0.119586, total 1.301468, recall: 0.95833, precision: 0.01411]
[Epoch 0/20, Batch 200/459] [Losses: x 0.225052, y 0.110916, w 0.037937, h 0.041618, conf 0.629934, cls 0.126029, total 1.171485, recall: 0.91667, precision: 0.01399]
[Epoch 0/20, Batch 201/459] [Losses: x 0.145981, y 0.273280, w 0.019811, h 0.130580, conf 0.434312, cls 0.118370, total 1.122333, recall: 1.00000, precision: 0.01511

[Epoch 0/20, Batch 245/459] [Losses: x 0.206383, y 0.200641, w 0.044117, h 0.172762, conf 0.603350, cls 0.475271, total 1.702523, recall: 0.00000, precision: 0.00000]
[Epoch 0/20, Batch 246/459] [Losses: x 0.321146, y 0.260715, w 0.146415, h 0.053945, conf 0.743348, cls 0.441946, total 1.967515, recall: 0.08333, precision: 0.00042]
[Epoch 0/20, Batch 247/459] [Losses: x 0.306680, y 0.286117, w 0.081803, h 0.183691, conf 0.768800, cls 0.439628, total 2.066720, recall: 0.04167, precision: 0.00009]
[Epoch 0/20, Batch 248/459] [Losses: x 0.131945, y 0.281984, w 0.084135, h 0.047048, conf 0.660788, cls 0.425939, total 1.631839, recall: 0.00000, precision: 0.00000]
[Epoch 0/20, Batch 249/459] [Losses: x 0.182714, y 0.193619, w 0.067530, h 0.144356, conf 0.584983, cls 0.425212, total 1.598415, recall: 0.00000, precision: 0.00000]
[Epoch 0/20, Batch 250/459] [Losses: x 0.208457, y 0.243539, w 0.129936, h 0.169094, conf 1.042823, cls 0.357408, total 2.151258, recall: 0.08333, precision: 0.00038